# 🌍 AI Travel Assistant with Multimodal Memory

This notebook demonstrates a production-ready AI travel assistant that processes multiple content types with persistent memory. You'll build an agent that remembers user preferences across sessions and provides personalized travel recommendations using [Strands Agents](https://strandsagents.com/) and [Amazon Bedrock](https://aws.amazon.com/bedrock/).

## What You'll Learn

- Build a conversational travel assistant with multimodal capabilities
- Process images, documents, and videos for travel planning
- Implement cross-session personalization with persistent memory
- Use semantic search for context-aware recommendations
- Handle real-world travel planning scenarios

## Prerequisites

- AWS account with [Amazon Bedrock access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
- Claude 3.5 Sonnet model enabled in Bedrock
- Vector memory backend (FAISS or Amazon S3 Vectors)
- Sample travel content (generated from notebook 05)

## Real-World Demo Scenario

You're planning a trip to Barcelona. Over multiple sessions, you:
1. Share your travel preferences
2. Show a photo of your destination
3. Upload your travel itinerary
4. Ask for personalized recommendations

The AI assistant remembers everything and provides increasingly personalized responses.


## Setup

### Prerequisites

1. Run `python travel_content_generator.py` to create demo assets
2. Configure your S3 Vector bucket (from notebook 04)
3. Ensure AWS credentials are configured

In [ ]:
import boto3
import os
from strands import Agent
from strands.models import BedrockModel
from strands_tools import image_reader, file_read
from video_reader_local import video_reader_local
from s3_memory import s3_vector_memory

print("✅ All imports successful!")

## Configuration

In [ ]:
# AWS Configuration
AWS_REGION = 'us-east-1'
os.environ['AWS_REGION'] = AWS_REGION

# Configure S3 Vectors (bucket and index will be created automatically)
os.environ['VECTOR_BUCKET_NAME'] = 'multimodal-vector-store'  # ⚠️ CHANGE THIS!
os.environ['VECTOR_INDEX_NAME'] = 'strands-multimodal'        # ⚠️ CHANGE THIS!
os.environ['EMBEDDING_MODEL'] = 'amazon.nova-2-multimodal-embeddings-v1:0'  # Nova embeddings


## Create Travel Assistant Agent

We'll create an agent with:
- **Vector memory** for cross-session persistence
- **Image analysis** for destination photos
- **Document processing** for itineraries
- **Personalization** based on stored preferences

In [ ]:
USER_ID = "demo_user_eli"  # Your user ID for memory isolation

print(f"🌍 Region: {AWS_REGION}")
print(f"📦 Vector bucket: {os.environ['VECTOR_BUCKET_NAME']}")
print(f"👤 User ID: {USER_ID}")

In [ ]:
# Setup Bedrock model

session = boto3.Session(region_name=AWS_REGION)
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# System prompt for travel assistant
TRAVEL_ASSISTANT_PROMPT = """You are an expert AI travel assistant with persistent memory.

Your capabilities:
- **Personalized Recommendations**: Tailor suggestions based on user preferences
- **Multimodal Analysis**: Process photos, documents, and text
- **Cross-Session Memory**: Remember preferences and context from previous conversations
- **Cultural Expertise**: Provide insights about destinations, cuisine, and local experiences

Memory Usage Guidelines:
1. **Always start** by retrieving relevant memories about the user
2. **Store important information**:
   - Travel preferences (food, activities, accommodation style)
   - Dietary restrictions or requirements
   - Budget considerations
   - Destinations of interest
   - Past travel experiences
3. **Build context** over multiple interactions
4. **Reference previous conversations** when relevant

When analyzing content:
1. Retrieve user's travel preferences from memory
2. Analyze the new content (photo, document, etc.)
3. Store key insights and details
4. Provide personalized recommendations based on both

Always be enthusiastic, helpful, and culturally sensitive.
"""

# Create travel assistant agent
travel_assistant = Agent(
    model=bedrock_model,
    tools=[
        s3_vector_memory,  # Persistent memory
        image_reader,      # Photo analysis
        video_reader_local,      # Video analysis
        file_read         # Document processing
    ],
    system_prompt=TRAVEL_ASSISTANT_PROMPT
)

print("✅ Travel Assistant Agent created!")

## Session 1: Establishing Travel Preferences

In the first session, the user shares their travel style and preferences.

In [ ]:
response = travel_assistant(
    f"""Hi! I'm planning my next trip and wanted to share my travel preferences with you.
    
    Here's what I love:
    - **Architecture**: I'm fascinated by modern architecture, especially Art Nouveau and Modernist styles
    - **Food**: I prefer gluthen free cuisine and love exploring local food markets
    - **Sustainability**: I try to travel sustainably - public transport, eco-friendly hotels, supporting local businesses
    - **Activities**: I enjoy walking tours, photography, and cultural experiences over beach/resort vacations
    - **Pace**: I prefer a relaxed pace with time to really experience each place
    
    Please remember these preferences for our future conversations.
    
    USER_ID: {USER_ID}"""
)

print(response)

## Session 2: Analyzing Destination Photo

A few days later, the user shares a photo of their dream destination.

**Simulating a new session** - the agent has no conversation history, only vector memory.

In [ ]:
image = "output/professional_travel_photography_of_alcatraz.png"
video = "output/san-francisco-tour.mp4"
document = "output/san-francisco-itinerary.txt"

In [ ]:
response = travel_assistant(
    f"""I found this amazing photo of a place I'd love to visit!
    
    Please analyze the image at: {image}
    
    USER_ID: {USER_ID}"""
)

print(response)

## Session 3: Processing Travel Itinerary

The user has booked the trip and shares their itinerary for personalized suggestions.

**Another new session** - testing cross-session memory again.

In [ ]:
# Clear conversation history again for another session
travel_assistant.messages.clear()

print("🆕 Another new session - conversation history cleared again")
print("🧠 Memory persists across all sessions!")
print(f"💬 Messages in history: {len(travel_assistant.messages)}")

In [ ]:
response = travel_assistant(
    f"""Great news! I've booked my trip and have my itinerary ready.
    
    Please review my itinerary: {document}
    
    USER_ID: {USER_ID}"""
)

print(response)

## Session 4: Personalized Recommendations

Days later, the user asks for restaurant recommendations.

**Final new session** - the agent should remember everything.

In [ ]:
# Clear conversation history one more time
travel_assistant.messages.clear()

print("🆕 Final new session - conversation history cleared")
print("🧠 Let's see if it remembers everything from vector memory!")
print(f"💬 Messages in history: {len(travel_assistant.messages)}")

In [ ]:
response = travel_assistant(
    f"""I'm leaving for my trip in a few days! 
    
    Can you recommend some restaurants I should try?
    
    USER_ID: {USER_ID}"""
)

print(response)

## Session 5: Video Analysis

The user shares a travel video they found online and wants personalized insights.

**Another new session** - demonstrating video analysis with memory context.

In [ ]:
# Clear conversation history for video analysis session
travel_assistant.messages.clear()

print("🆕 New session for video analysis")
print("🧠 Agent will use memory to provide personalized video insights")
print(f"💬 Messages in history: {len(travel_assistant.messages)}")

In [ ]:
response = travel_assistant(
    f"""I found this video I'd love your thoughts!
    
    Please analyze the video at: {video}
    
    USER_ID: {USER_ID}"""
)

print(response)

## Inspecting the Memory

Let's see what the agent has stored in its vector memory.

In [ ]:
# List all memories
result = s3_vector_memory(
    action="list",
    user_id=USER_ID,
    top_k=20
)

print(f"📊 Total memories stored: {result['total_found']}")
print("\n" + "="*80)
print("Stored Memories:")
print("="*80)

for i, mem in enumerate(result.get('memories', []), 1):
    content = mem.get('memory', '')
    timestamp = mem.get('created_at', 'N/A')
    
    print(f"\n{i}. [{timestamp}]")
    print(f"   {content[:200]}..." if len(content) > 200 else f"   {content}")
    print("-" * 80)

## Semantic Memory Search

Test the semantic search capabilities.

In [ ]:
# Search for architecture-related memories
result = s3_vector_memory(
    action="retrieve",
    query="architectural preferences",
    user_id=USER_ID,
    top_k=5
)

print("🔍 Search: 'architectural preferences and Gaudí buildings'")
print("\nTop Results:")
for i, mem in enumerate(result.get('memories', []), 1):
    print(f"\n{i}. Similarity: {mem.get('similarity', 'N/A')}")
    print(f"   {mem.get('memory', '')}")

In [ ]:
# Search for food-related memories
result = s3_vector_memory(
    action="retrieve",
    query="food preferences and restaurants",
    user_id=USER_ID,
    top_k=5
)

print("🔍 Search: 'vegetarian food preferences and restaurants'")
print("\nTop Results:")
for i, mem in enumerate(result.get('memories', []), 1):
    print(f"\n{i}. Similarity: {mem.get('similarity', 'N/A')}")
    print(f"   {mem.get('memory', '')}")

### Memory Flow

**Storing Memory:**
1. Agent extracts key information from conversation
2. Calls `s3_vector_memory(action="store", content=..., user_id=...)`
3. Text is converted to embedding via Nova
4. Embedding + metadata stored in S3 Vectors

**Retrieving Memory:**
1. Agent needs context for response
2. Calls `s3_vector_memory(action="retrieve", query=..., user_id=...)`
3. Query converted to embedding
4. Similarity search in S3 Vectors
5. Top-K most relevant memories returned
6. Agent uses memories to inform response

### Cost Optimization

- **Embeddings**: ~$0.0001 per 1K tokens (Nova)
- **Storage**: S3 Vectors pricing (cost-optimized)
- **Queries**: Fast sub-second retrieval
- **Tip**: Use appropriate `top_k` values to balance cost and relevance

## Summary

In this demo, you learned:

✅ How to build a personalized AI assistant with persistent memory

✅ Cross-session context retention without conversation history

✅ Multimodal content analysis (text, images, PDFs)

✅ Semantic memory search and retrieval

✅ Progressive personalization over multiple interactions

✅ Production-ready architecture with AWS services

### Resources

- [Amazon S3 Vectors Documentation](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-vectors.html)
- [Amazon Nova Embeddings](https://aws.amazon.com/blogs/aws/amazon-nova-multimodal-embeddings-now-available-in-amazon-bedrock/)
- [Strands Agents SDK](https://github.com/awslabs/strands)

Happy building! 🚀